In [2]:
import os

from tpu_graphs.baselines.layout.data import get_npz_dataset

In [ ]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"

dataset = get_npz_dataset(
    root_path=
)